In [1]:
import numpy as np
import os,cv2,math,glob,random
import scipy.io as sio
from math import cos, sin
from pathlib import Path
import pandas as pd
import mediapipe
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Loading AFLW2000 Data
%%capture
if os.path.isfile('AFLW2000-3D.zip') == False:
  !gdown --id  1fP3zvSCYjll_o_m7S12nvQLZ9MnsEoap
  !unzip /content/AFLW2000-3D.zip

UsageError: Line magic function `%%capture` not found.


In [4]:
def draw_axis(img, pitch,yaw,roll, tdx=None, tdy=None, size = 100):

    yaw = -yaw
    if tdx != None and tdy != None:
        tdx = tdx
        tdy = tdy
    else:
        height, width = img.shape[:2]
        tdx = width / 2
        tdy = height / 2

    # X-Axis pointing to right. drawn in red
    x1 = size * (cos(yaw) * cos(roll)) + tdx
    y1 = size * (cos(pitch) * sin(roll) + cos(roll) * sin(pitch) * sin(yaw)) + tdy

    # Y-Axis | drawn in green
    #        v
    x2 = size * (-cos(yaw) * sin(roll)) + tdx
    y2 = size * (cos(pitch) * cos(roll) - sin(pitch) * sin(yaw) * sin(roll)) + tdy

    # Z-Axis (out of the screen) drawn in blue
    x3 = size * (sin(yaw)) + tdx
    y3 = size * (-cos(yaw) * sin(pitch)) + tdy

    cv2.line(img, (int(tdx), int(tdy)), (int(x1),int(y1)),(0,0,255),3)
    cv2.line(img, (int(tdx), int(tdy)), (int(x2),int(y2)),(0,255,0),3)
    cv2.line(img, (int(tdx), int(tdy)), (int(x3),int(y3)),(255,0,0),2)

    return img

In [5]:
random_image = sorted([Path(f).stem for f in glob.glob("AFLW2000/*.mat")])

In [6]:
DataSetArray=[]
for j in range(468):
    DataSetArray.append("x"+str(j))
    DataSetArray.append("y"+str(j))
DataSetArray.append("Pitch")
DataSetArray.append("Yaw")
DataSetArray.append("Roll")
Data=pd.DataFrame(columns=DataSetArray)

In [7]:
Data

,x0,y0,x1,y1,x2,y2,x3,y3,x4,y4,...,y464,x465,y465,x466,y466,x467,y467,Pitch,Yaw,Roll


In [8]:
j =0
for i in random_image:
    values = {}
    faceModule = mediapipe.solutions.face_mesh
    # loading image and its correspinding mat file
    with faceModule.FaceMesh(static_image_mode=True) as faces:
        # loading the image
        image = cv2.imread('AFLW2000/'+i+'.jpg')
        # processing the face to extract the landmark points (468 point) for each x,y,z
        results = faces.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        if results.multi_face_landmarks != None: 
            # looping over the faces in the image
            for face in results.multi_face_landmarks:
                k = 0
                for landmark in face.landmark:
                    x = landmark.x
                    y = landmark.y
                    # note: the x and y values are scaled to the their width and height so we will get back their actual value in the image
                    shape = image.shape 
                    relative_x = int(x * shape[1])
                    values['x'+str(k)] = relative_x
                    relative_y = int(y * shape[0])
                    values['y'+str(k)] = relative_y
                    k = k+1
            # loading the mat file to extract the 3 angels of the face (pitch,yaw,roll)
            mat_file = sio.loadmat('AFLW2000/'+i+'.mat')
            # extracting the labels 3 angels
            pose_para = mat_file["Pose_Para"][0][:3]
            pitch = pose_para[0]
            values['Pitch'] = pitch
            yaw = pose_para[1]
            values['Yaw'] = yaw
            roll = pose_para[2]
            values['Roll'] = roll
            Data = Data.append(pd.DataFrame([values]),ignore_index=True)
            j = j+1
            print(j)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [9]:
Data_copy = pd.DataFrame.copy(Data)
Data

,x0,y0,x1,y1,x2,y2,x3,y3,x4,y4,...,y464,x465,y465,x466,y466,x467,y467,Pitch,Yaw,Roll
0,218,309,220,287,220,291,215,253,220,279,...,221,243,225,299,212,304,208,-0.399231,0.018227,0.085676
1,198,288,187,266,201,274,196,245,187,259,...,230,225,231,263,225,267,222,0.470065,1.189533,0.300959
2,143,359,144,349,143,351,139,335,144,345,...,324,147,325,168,320,169,319,-0.18465,0.881137,-0.236852
3,226,312,211,294,220,296,199,265,207,286,...,229,220,233,268,205,271,200,-0.175379,0.299208,-0.373374
4,229,301,221,272,224,281,210,244,219,263,...,219,227,221,271,203,275,201,-0.026812,0.011965,-0.220662
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1848,231,307,239,281,233,287,230,252,240,273,...,226,243,228,283,215,286,213,-0.306358,-0.283822,0.038554
1849,244,310,254,285,247,291,245,253,255,277,...,225,258,227,296,215,300,211,-0.367547,-0.429723,0.122791
1850,263,306,271,282,267,290,271,255,274,274,...,242,292,243,330,246,334,243,-0.156035,0.567114,-0.108536
1851,221,311,225,286,223,292,220,251,226,277,...,223,247,226,303,216,308,213,-0.197102,-0.07043,0.105118


In [10]:
Data = Data_copy
Data

,x0,y0,x1,y1,x2,y2,x3,y3,x4,y4,...,y464,x465,y465,x466,y466,x467,y467,Pitch,Yaw,Roll
0,218,309,220,287,220,291,215,253,220,279,...,221,243,225,299,212,304,208,-0.399231,0.018227,0.085676
1,198,288,187,266,201,274,196,245,187,259,...,230,225,231,263,225,267,222,0.470065,1.189533,0.300959
2,143,359,144,349,143,351,139,335,144,345,...,324,147,325,168,320,169,319,-0.18465,0.881137,-0.236852
3,226,312,211,294,220,296,199,265,207,286,...,229,220,233,268,205,271,200,-0.175379,0.299208,-0.373374
4,229,301,221,272,224,281,210,244,219,263,...,219,227,221,271,203,275,201,-0.026812,0.011965,-0.220662
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1848,231,307,239,281,233,287,230,252,240,273,...,226,243,228,283,215,286,213,-0.306358,-0.283822,0.038554
1849,244,310,254,285,247,291,245,253,255,277,...,225,258,227,296,215,300,211,-0.367547,-0.429723,0.122791
1850,263,306,271,282,267,290,271,255,274,274,...,242,292,243,330,246,334,243,-0.156035,0.567114,-0.108536
1851,221,311,225,286,223,292,220,251,226,277,...,223,247,226,303,216,308,213,-0.197102,-0.07043,0.105118


In [11]:
p1 = Data['x1']
p2 = Data['y1']
p3 = (((Data['x200']-Data['x109'])**2)+((Data['y200']-Data['y109'])**2))**0.5
for col in Data.columns:
    if col[0] == 'x':
        Data[col] = (Data[col] - p1)/p3
    if col[0] == 'y':
        Data[col] = (Data[col] - p2)/p3
    else:
        pass

In [12]:
Data

,x0,y0,x1,y1,x2,y2,x3,y3,x4,y4,...,y464,x465,y465,x466,y466,x467,y467,Pitch,Yaw,Roll
0,-0.008914,0.098049,0.0,0.0,0.0,0.017827,-0.022284,-0.151531,0.0,-0.035654,...,-0.294148,0.102506,-0.276321,0.352086,-0.334259,0.37437,-0.352086,-0.399231,0.018227,0.085676
1,0.077457,0.154914,0.0,0.0,0.098582,0.056332,0.063374,-0.147873,0.0,-0.049291,...,-0.253496,0.267579,-0.246454,0.535158,-0.288704,0.563324,-0.309828,0.470065,1.189533,0.300959
2,-0.01096,0.109599,0.0,0.0,-0.01096,0.02192,-0.0548,-0.153439,0.0,-0.04384,...,-0.273998,0.03288,-0.263038,0.263038,-0.317838,0.273998,-0.328798,-0.18465,0.881137,-0.236852
3,0.081801,0.098162,0.0,0.0,0.049081,0.010907,-0.065441,-0.158149,-0.021814,-0.043627,...,-0.354472,0.049081,-0.332659,0.310845,-0.485354,0.327205,-0.512621,-0.175379,0.299208,-0.373374
4,0.040283,0.146026,0.0,0.0,0.015106,0.045318,-0.055389,-0.14099,-0.010071,-0.045318,...,-0.266875,0.030212,-0.256804,0.251769,-0.347441,0.27191,-0.357511,-0.026812,0.011965,-0.220662
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1848,-0.041464,0.134758,0.0,0.0,-0.031098,0.031098,-0.046647,-0.150307,0.005183,-0.041464,...,-0.285066,0.020732,-0.2747,0.228053,-0.342079,0.243602,-0.352445,-0.306358,-0.283822,0.038554
1849,-0.048264,0.120659,0.0,0.0,-0.033785,0.028958,-0.043437,-0.154443,0.004826,-0.038611,...,-0.289581,0.019305,-0.279929,0.202707,-0.337845,0.222012,-0.357151,-0.367547,-0.429723,0.122791
1850,-0.044117,0.132352,0.0,0.0,-0.022059,0.044117,0.0,-0.148897,0.016544,-0.044117,...,-0.220587,0.115808,-0.215073,0.325367,-0.198529,0.347425,-0.215073,-0.156035,0.567114,-0.108536
1851,-0.018006,0.11254,0.0,0.0,-0.009003,0.027009,-0.022508,-0.157555,0.004502,-0.040514,...,-0.2836,0.099035,-0.270095,0.351123,-0.315111,0.373631,-0.328616,-0.197102,-0.07043,0.105118


In [13]:
Data

,x0,y0,x1,y1,x2,y2,x3,y3,x4,y4,...,y464,x465,y465,x466,y466,x467,y467,Pitch,Yaw,Roll
0,-0.008914,0.098049,0.0,0.0,0.0,0.017827,-0.022284,-0.151531,0.0,-0.035654,...,-0.294148,0.102506,-0.276321,0.352086,-0.334259,0.37437,-0.352086,-0.399231,0.018227,0.085676
1,0.077457,0.154914,0.0,0.0,0.098582,0.056332,0.063374,-0.147873,0.0,-0.049291,...,-0.253496,0.267579,-0.246454,0.535158,-0.288704,0.563324,-0.309828,0.470065,1.189533,0.300959
2,-0.01096,0.109599,0.0,0.0,-0.01096,0.02192,-0.0548,-0.153439,0.0,-0.04384,...,-0.273998,0.03288,-0.263038,0.263038,-0.317838,0.273998,-0.328798,-0.18465,0.881137,-0.236852
3,0.081801,0.098162,0.0,0.0,0.049081,0.010907,-0.065441,-0.158149,-0.021814,-0.043627,...,-0.354472,0.049081,-0.332659,0.310845,-0.485354,0.327205,-0.512621,-0.175379,0.299208,-0.373374
4,0.040283,0.146026,0.0,0.0,0.015106,0.045318,-0.055389,-0.14099,-0.010071,-0.045318,...,-0.266875,0.030212,-0.256804,0.251769,-0.347441,0.27191,-0.357511,-0.026812,0.011965,-0.220662
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1848,-0.041464,0.134758,0.0,0.0,-0.031098,0.031098,-0.046647,-0.150307,0.005183,-0.041464,...,-0.285066,0.020732,-0.2747,0.228053,-0.342079,0.243602,-0.352445,-0.306358,-0.283822,0.038554
1849,-0.048264,0.120659,0.0,0.0,-0.033785,0.028958,-0.043437,-0.154443,0.004826,-0.038611,...,-0.289581,0.019305,-0.279929,0.202707,-0.337845,0.222012,-0.357151,-0.367547,-0.429723,0.122791
1850,-0.044117,0.132352,0.0,0.0,-0.022059,0.044117,0.0,-0.148897,0.016544,-0.044117,...,-0.220587,0.115808,-0.215073,0.325367,-0.198529,0.347425,-0.215073,-0.156035,0.567114,-0.108536
1851,-0.018006,0.11254,0.0,0.0,-0.009003,0.027009,-0.022508,-0.157555,0.004502,-0.040514,...,-0.2836,0.099035,-0.270095,0.351123,-0.315111,0.373631,-0.328616,-0.197102,-0.07043,0.105118


In [14]:
from sklearn.model_selection import train_test_split

In [15]:
Train_Data , Test_Data = train_test_split(Data, test_size=0.15, random_state=42)

In [16]:
# Training Data split
Train_Features=Train_Data.iloc[:,:936]
Train_pitch=Train_Data.iloc[:,-3]
Train_yaw=Train_Data.iloc[:,-2]
Train_roll=Train_Data.iloc[:,-1]

In [17]:
# Testing Data split
Test_Features=Test_Data.iloc[:,:936]
Test_pitch=Test_Data.iloc[:,-3]
Test_yaw=Test_Data.iloc[:,-2]
Test_roll=Test_Data.iloc[:,-1]

In [18]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
parameters1 = {'kernel':['rbf'], 'C':[0.01,0.10,1, 10],'gamma':[0.01,0.10,1, 10]}
parameters = {'kernel':['linear'],'C':[0.01,0.10,1, 10]}

In [19]:
# Grid Search for pitch
svm = SVR()
pitch_svr_gs = GridSearchCV(estimator = svm,param_grid = parameters,scoring='neg_mean_absolute_error')
pitch_svr_gs.fit(Train_Features, Train_pitch)

GridSearchCV(estimator=SVR(),
             param_grid={'C': [0.01, 0.1, 1, 10], 'kernel': ['linear']},
             scoring='neg_mean_absolute_error')

In [20]:
pitch_svm_winner = pitch_svr_gs.best_estimator_
print(pitch_svm_winner)
print(pitch_svm_winner.score(Test_Features, Test_pitch))

SVR(C=10, kernel='linear')
0.7358706218780819


In [21]:
from sklearn.metrics import mean_absolute_error
print(mean_absolute_error(Test_pitch,pitch_svm_winner.predict(Test_Features)))
print(mean_absolute_error(Train_pitch,pitch_svm_winner.predict(Train_Features)))

0.08567627930232824
0.10369305594682453


In [22]:
# Grid Search for yaw
parameters2={'kernel':['poly'],'degree':[3]}
svm1 = SVR()
yaw_svr_gs = GridSearchCV(estimator = svm1 ,param_grid = parameters2,scoring='neg_mean_absolute_error')
yaw_svr_gs.fit(Train_Features, Train_yaw)

GridSearchCV(estimator=SVR(), param_grid={'degree': [3], 'kernel': ['poly']},
             scoring='neg_mean_absolute_error')

In [23]:
yaw_svm_winner = yaw_svr_gs.best_estimator_
print(yaw_svm_winner)
print(yaw_svm_winner.score(Test_Features, Test_yaw))

SVR(kernel='poly')
0.9622952938237667


In [24]:
from sklearn.metrics import mean_absolute_error
print(mean_absolute_error(Test_yaw,yaw_svm_winner.predict(Test_Features)))
print(mean_absolute_error(Train_yaw,yaw_svm_winner.predict(Train_Features)))

0.062337221613087294
0.07338049617002917


In [25]:
# Grid Search for roll
parameters3={'kernel':['poly'],'degree':[3]}
svm2 = SVR()
roll_svr_gs = GridSearchCV(estimator = svm2,param_grid = parameters,scoring='neg_mean_absolute_error')
roll_svr_gs.fit(Train_Features, Train_roll)

GridSearchCV(estimator=SVR(),
             param_grid={'C': [0.01, 0.1, 1, 10], 'kernel': ['linear']},
             scoring='neg_mean_absolute_error')

In [26]:
roll_svm_winner = roll_svr_gs.best_estimator_
print(roll_svm_winner)
print(roll_svm_winner.score(Test_Features, Test_roll))

SVR(C=10, kernel='linear')
0.7157321416506495


In [27]:
from sklearn.metrics import mean_absolute_error
print(mean_absolute_error(Test_roll,roll_svm_winner.predict(Test_Features)))
print(mean_absolute_error(Train_roll,roll_svm_winner.predict(Train_Features)))

0.07134284350485186
0.08623551244615876


In [51]:
# Reading testing Video and split it into frames

In [52]:
vidcap = cv2.VideoCapture('New_Project.mp4')
success,video_image = vidcap.read()

In [53]:
DataSetArray1=[]
for j in range(468):
  DataSetArray1.append("x"+str(j))
  DataSetArray1.append("y"+str(j))
Data1=pd.DataFrame(columns=DataSetArray1)

In [54]:
#for i in video_image[:2]:
vidcap = cv2.VideoCapture('HPE.mp4')
width= int(vidcap.get(cv2.CAP_PROP_FRAME_WIDTH))
height= int(vidcap.get(cv2.CAP_PROP_FRAME_HEIGHT))
s=0
image_use=[]
xaxis=[]
yaxis=[]
if (vidcap.isOpened()== False): 
    print("Error opening file")

while(vidcap.isOpened):
  success,video_image = vidcap.read()
  current_image_test={}

  faceModule = mediapipe.solutions.face_mesh
  # loading image and its correspinding mat file
  if success == True:
    with faceModule.FaceMesh(static_image_mode=True) as faces:
        # loading the image
        image1 = video_image
        
        # processing the face to extract the landmark points (468 point) for each x,y,z
        results = faces.process(cv2.cvtColor(image1, cv2.COLOR_BGR2RGB))
        if results.multi_face_landmarks != None: 
            image_use.append(image1)
            # looping over the faces in the image
            for face in results.multi_face_landmarks:
                k = 0
                for landmark in face.landmark:
                    x = landmark.x
                    y = landmark.y
                    
                    if k == 217 :
                      xaxis.append(x)
                      yaxis.append(y)
                    # note: the x and y values are scaled to the their width and height so we will get back their actual value in the image
                    shape = image1.shape 
                    relative_x = int(x * shape[1])
                    relative_y = int(y * shape[0])
                    #Add relative x & relative y to current list current_image_test
                    current_image_test['x'+str(k)]=relative_x
                    current_image_test['y'+str(k)]=relative_y
                    # cv2.putText(image, str(relative_y), (int(relative_x),int(relative_y)), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0,255,0), 2)
                    #cv2.circle(image1, (relative_x, relative_y), radius=1, color=(0, 255, 0), thickness=2)
                    k+=1

            print(s)
            s+=1
            Data1 = pd.concat([Data1, pd.DataFrame([current_image_test])], axis=0)
  # Break the loop
  else: 
      break
vidcap.release()  
# Closes all the frames
cv2.destroyAllWindows()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [55]:
Data1_copy = pd.DataFrame.copy(Data1)
Data1

,x0,y0,x1,y1,x2,y2,x3,y3,x4,y4,...,x463,y463,x464,y464,x465,y465,x466,y466,x467,y467
0,338,165,337,147,337,153,331,131,337,142,...,346,119,343,120,342,120,369,116,371,115
0,338,165,337,148,337,153,331,131,337,142,...,346,119,343,120,342,120,369,116,371,115
0,338,166,337,148,337,154,331,132,337,143,...,346,119,343,120,342,121,369,117,371,116
0,338,166,337,148,337,154,331,131,337,143,...,346,119,343,120,342,121,369,117,371,115
0,338,166,337,148,337,154,332,131,337,143,...,346,119,343,120,342,121,369,116,371,115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,350,171,350,153,350,158,344,137,350,148,...,359,123,356,124,355,125,382,119,384,117
0,350,171,349,153,349,158,343,136,349,147,...,358,122,355,123,354,124,382,118,384,117
0,349,170,348,151,348,157,343,134,348,146,...,357,121,354,123,353,123,380,118,382,117
0,349,168,348,150,348,155,342,132,348,144,...,357,120,354,121,353,121,380,116,383,115


In [56]:
p1 = Data1['x1']
p2 = Data1['y1']
p3 = (((Data1['x200']-Data1['x109'])**2)+((Data1['y200']-Data1['y109'])**2))**0.5
for col in Data.columns:
    if col[0] == 'x':
        Data1[col] = (Data1[col] - p1)/p3
    if col[0] == 'y':
        Data1[col] = (Data1[col] - p2)/p3
    else:
        pass

In [57]:
Data1

,x0,y0,x1,y1,x2,y2,x3,y3,x4,y4,...,x463,y463,x464,y464,x465,y465,x466,y466,x467,y467
0,0.008822,0.158789,0.0,0.0,0.0,0.05293,-0.05293,-0.141146,0.0,-0.044108,...,0.079395,-0.247005,0.05293,-0.238184,0.044108,-0.238184,0.282292,-0.27347,0.299935,-0.282292
0,0.008822,0.149968,0.0,0.0,0.0,0.044108,-0.05293,-0.149968,0.0,-0.05293,...,0.079395,-0.255827,0.05293,-0.247005,0.044108,-0.247005,0.282292,-0.282292,0.299935,-0.291114
0,0.008746,0.157429,0.0,0.0,0.0,0.052476,-0.052476,-0.139937,0.0,-0.04373,...,0.078715,-0.253636,0.052476,-0.24489,0.04373,-0.236144,0.279874,-0.271128,0.297366,-0.279874
0,0.008746,0.157429,0.0,0.0,0.0,0.052476,-0.052476,-0.148683,0.0,-0.04373,...,0.078715,-0.253636,0.052476,-0.24489,0.04373,-0.236144,0.279874,-0.271128,0.297366,-0.28862
0,0.008746,0.157429,0.0,0.0,0.0,0.052476,-0.04373,-0.148683,0.0,-0.04373,...,0.078715,-0.253636,0.052476,-0.24489,0.04373,-0.236144,0.279874,-0.279874,0.297366,-0.28862
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0.0,0.166268,0.0,0.0,0.0,0.046186,-0.055423,-0.147794,0.0,-0.046186,...,0.083134,-0.277113,0.055423,-0.267876,0.046186,-0.258639,0.295588,-0.314062,0.314062,-0.332536
0,0.008898,0.160173,0.0,0.0,0.0,0.044492,-0.053391,-0.151274,0.0,-0.053391,...,0.080086,-0.275853,0.053391,-0.266954,0.044492,-0.258056,0.29365,-0.311447,0.311447,-0.320345
0,0.008672,0.164763,0.0,0.0,0.0,0.052031,-0.043359,-0.14742,0.0,-0.043359,...,0.078046,-0.260153,0.052031,-0.242809,0.043359,-0.242809,0.277496,-0.286168,0.29484,-0.29484
0,0.008584,0.154508,0.0,0.0,0.0,0.042919,-0.051503,-0.154508,0.0,-0.051503,...,0.077254,-0.257513,0.051503,-0.248929,0.042919,-0.248929,0.274681,-0.291848,0.300432,-0.300432


In [58]:
pitch_prediction = pitch_svm_winner.predict(Data1)
yaw_prediction = yaw_svm_winner.predict(Data1)
roll_prediction = roll_svm_winner.predict(Data1)

In [59]:
len(image_use)

548

In [60]:
xt=np.array(Data1_copy['x1'])
yt=np.array(Data1_copy['y1'])

In [61]:
final_video_image=[]
for i in range(0,len(image_use)):
  z=draw_axis(image_use[i], pitch_prediction[i],yaw_prediction[i],roll_prediction[i],xt[i],yt[i])
  final_video_image.append(z)

In [62]:
out = cv2.VideoWriter('oubBt.mp4',cv2.VideoWriter_fourcc(*'DIVX'), 20, (width,height))
for i in range(len(final_video_image)):
  out.write(final_video_image[i])
out.release()